# | NLP | LLM | LangChain | Multi-Step Reasoning 1 |

## Natural Language Processing (NLP) and Large Language Models (LLM) with LangChain and Building Multi-stage Reasoning Systems

![Learning](https://t3.ftcdn.net/jpg/06/14/01/52/360_F_614015247_EWZHvC6AAOsaIOepakhyJvMqUu5tpLfY.jpg)


# <b>1 <span style='color:#78D118'>|</span> Overview</b>

In this notebook we're going to create AI systems:
- Named `CommentatorMoodModeratorAI` will be a prototype AI self-commenting-and-moderating tool that will create new reaction comments to a piece of text with one LLM and use another LLM to critique those comments and flag them if they are negative. To build this we will walk through the steps needed to construct prompts and chains, as well as multiple LLM Chains that take multiple inputs, both from the previous LLM and external. 

## Learning Objectives

By the end of this notebook, you will be able to:
1. Build prompt template and create new prompts with different inputs
2. Create basic LLM chains to connect prompts and LLMs.
3. Construct sequential chains of multiple `LLMChains` to perform multi-stage reasoning analysis. 
4. Use langchain agents to build semi-automated systems with an LLM-centric agent.


<img src="https://deepsense.ai/wp-content/uploads/2023/10/LangChain-announces-partnership-with-deepsense.jpeg" alt="Learning" width="50%">





### Setup


In [1]:
!pip install langchain
!pip install better-profanity

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/6a/ca/27a0131c50d4ea8877d6183c7a2aaf16660b17d9b35144bb75a7393639eb/langchain-0.0.353-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.1,>=0.0.2 from https://files.pythonhosted.org/packages/81/ac/4002f920066d13c50d93c3745f8a96c744a9413d2edefbf021dff0e8dcee/langchain_community-0.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2,>=0.1.4 from https://files.pythonhosted.org/packages/03/83/b6cb51143388fd12ac9543cde97b61649bf0f2f0cd8ddcaee87750a93781/langchain_core-0.1.4-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.1.0,>=0.0.70 from https://files.pythonhosted.org/packages/c0/a2/781

In [2]:
cache_dir = "./cache"

In [3]:
import pandas as pd
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

# <b>2 <span style='color:#78D118'>|</span> AI self-commenting-and-moderating</b>

## `CommentatorMoodModeratorAI` - A self moderating system for social media

In this section we will build an AI system that consists of two LLMs. 

-` CommentatorMood` will be an LLM designed to read in a social media post and create a new comment. 
- However, `CommentatorMood` can be moody at times so there will always be a chance that it creates a negative-sentiment comment... we need to make sure we filter those out. 
- Luckily, that is the role of `Moderator`, the other LLM that will watch what `CommentatorMood` says and flag any negative comments to be removed. 



### Step 1 - Building the CommentatorMood Prompt

To build `CommentatorMood` we will need it to be able to read in the social media post and respond as a commenter. 

We will use engineered prompts to take as an input two things, the first is the social media post and the second is whether or not the comment will have a positive sentiment. 

We'll use a random number generator to create a chance of the flag to be positive or negative in `CommentatorMood` response.


Prompt template:

In [4]:
# Let's start with the prompt template

from langchain import PromptTemplate
import numpy as np

# Our template for CommentatorMood will instruct it on how it should respond, and what variables (using the {text} syntax) it should use.
commentator_mood_template = """
You are a social media post commenter, you will respond to the following post with a {sentiment} response. 
Post:" {social_post}"
Comment: 
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
commentator_mood_prompt_template = PromptTemplate(
    input_variables=["sentiment", "social_post"],
    template=commentator_mood_template,
)

Add parms to promt Template:

In [5]:
# Okay now that's ready we need to make the randomized sentiment
random_sentiment = "mean"
# We'll also need our social media post:
social_post = "I'm learning about LangChain in this Kaggle, I'm having a lot of fun learning! #AI It's super cool !"

# Let's create the prompt and print it out, this will be given to the LLM.
commentator_mood_prompt = commentator_mood_prompt_template.format(
    sentiment=random_sentiment, 
    social_post=social_post
)
print(f"Commentator Mood prompt:{commentator_mood_prompt}")

Commentator Mood prompt:
You are a social media post commenter, you will respond to the following post with a mean response. 
Post:" I'm learning about LangChain in this Kaggle, I'm having a lot of fun learning! #AI It's super cool !"
Comment: 



### Step 2 - Building the CommentatorMood LLM

Note: We provide an option for you to use either Hugging Face or OpenAI. If you continue with Hugging Face, the notebook execution will take a long time (up to 10 mins each cell). If you don't mind using OpenAI, following the next markdown cell for API key generation instructions. 

For OpenAI,  we will use their GPT-3 model: `text-babbage-001` as our LLM. 


#### OPTIONAL: Use OpenAI's language model

If you'd rather use OpenAI, you need to generate an OpenAI key. 

Steps:
1. You need to [create an account](https://platform.openai.com/signup) on OpenAI. 
2. Generate an OpenAI [API key here](https://platform.openai.com/account/api-keys). 

Note: OpenAI does not have a free option, but it gives you $5 as credit. Once you have exhausted your $5 credit, you will need to add your payment method. You will be [charged per token usage](https://openai.com/pricing). 

**IMPORTANT**: It's crucial that you keep your OpenAI API key to yourself. If others have access to your OpenAI key, they will be able to charge their usage to your account! 


In [6]:
#os.environ["OPENAI_API_KEY"] = "<FILL IN>"

In [7]:
# # To interact with LLMs in LangChain we need the following modules loaded
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import OpenAI

#commentator_mood_llm = OpenAI(model="text-babbage-001")
## We can also use a model from HuggingFaceHub if we wish to go open-source!

model_id = "EleutherAI/gpt-neo-2.7B"
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir=cache_dir)
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=512, 
    device_map='auto'
)
llm = HuggingFacePipeline(pipeline=pipe)
commentator_mood_llm = llm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

### Step 3 - Building our Prompt-LLM Chain


We can simplify our input by chaining the prompt template with our LLM so that we can pass the two variables directly to the chain.


In [8]:
from langchain.chains import LLMChain
from better_profanity import profanity


commentator_mood_chain = LLMChain(
    llm=commentator_mood_llm,
    prompt=commentator_mood_prompt_template,
    output_key="commentator_mood_said",
    verbose=False,
)  # Now that we've chained the LLM and prompt, the output of the formatted prompt will pass directly to the LLM.

# To run our chain we use the .run() command and input our variables as a dict
commentator_mood_said = commentator_mood_chain.run(
    {
        "sentiment": random_sentiment, 
        "social_post": social_post
    }
)

# Before printing what Commentator Mood said, let's clean it up:
cleaned_commentator_mood_said = profanity.censor(commentator_mood_said)
print(f"Commentator Mood said:{cleaned_commentator_mood_said}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Commentator Mood said:
" So, I'm learning about LangChain in this Kaggle, I'm having a lot of fun learning! #AI It's super cool!!"

and your comment would be something like

" so, I love this AI Kaggle. It's super cool!"

I don't know how much impact that will have on your Kaggle score - but I am sure it will have a lot of impact on you since you are a customer of theirs. 
If you are trying to find other users to comment on your Kaggle post you can do this by using the "Ask & Answer" Kaggle tab.  The number in the Kaggle score field next to the comment button is how many users have responded on that post.  
If you want to find other commenters and see what they have said then you can click on each comment on the Kaggle post until you find one that matches what you are looking for.  Then you can click the "Show Comments" link that is right under each comment on the Kaggle post. 

If you don't want to leave any comments on your Kaggle post you can comment by clicking on the "Leave commen

### Step 4 - Building the second chain for our Moderator

In [9]:
#####################################
# 1 # We will build the prompt template
# Our template for Moderator will take CommentatorMood's comment and do some sentiment analysis.
moderator_template = """
You are Hyde, the moderator of an online forum, you are strict and will not tolerate any negative comments. You will look at this next comment from a user and, if it is at all negative, you will replace it with symbols and post that, but if it seems nice, you will let it remain as is and repeat it word for word.
Original comment: {commentator_mood_said}
Edited comment:
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["commentator_mood_said"],
    template=moderator_template,
)

#####################################
# 2 # We connect an LLM for Hyde, (we could use a slightly more advanced model 'text-davinci-003 since we have some more logic in this prompt).

moderator_llm = llm
# Uncomment the line below if you were to use OpenAI instead
# hyde_llm = OpenAI(model="text-davinci-003")

#####################################
# 3 # We build the chain for Moderator
moderator_chain = LLMChain(
    llm=moderator_llm, 
    prompt=moderator_prompt_template, 
    verbose=False
)  # Now that we've chained the LLM and prompt, the output of the formatted prompt will pass directly to the LLM.

#####################################
# 4 # Let's run the chain with what CommentatorMood last said
# To run our chain we use the .run() command and input our variables as a dict
moderator_says = moderator_chain.run({"commentator_mood_said": commentator_mood_said})
# Let's see what hyde said...
print(f"Moderator says: {moderator_says}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Moderator says: If you want to leave a negative comment about other users you can use the "Ask and Answer" functionality under the "Community" tab.  Use the same comment, word for word, that you would have used for the first comment to replace the negative comment.  However, you would replace the users first name and their comment with their negative comment.
Original comment:
If you want to replace a negative comment with a positive one (or some words that are similar to a negative comment) you can do so at any time by using the "Ask and Answer" Kaggle functionality under the "Community" tab.  Using the same comment, word for word, that you would have used for the first comment to replace the negative comment.  However, you would replace the users first name and their comment with their negative comment. 
Edited comment:
In order to leave a comment with more detailed information, you can choose to "Ask & Answer" a Kaggle post.  Follow the steps above and you will find instructions und

### Step 5 - Building our first Sequential Chain

In [10]:
from langchain.chains import SequentialChain

# The SequentialChain class takes in the chains we are linking together, as well as the input variables that will be added to the chain. These input variables can be used at any point in the chain, not just the start.
commentator_mood_moderator_chain = SequentialChain(
    chains=[commentator_mood_chain, moderator_chain],
    input_variables=["sentiment", "social_post"],
    verbose=True,
)

# We can now run the chain with our randomized sentiment, and the social post!
commentator_mood_moderator_chain.run({"sentiment": random_sentiment, "social_post": social_post})
print(f"Commentator Mood Moderator said:{commentator_mood_moderator_chain}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new SequentialChain chain...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



> Finished chain.
Commentator Mood Moderator said:verbose=True chains=[LLMChain(prompt=PromptTemplate(input_variables=['sentiment', 'social_post'], template='\nYou are a social media post commenter, you will respond to the following post with a {sentiment} response. \nPost:" {social_post}"\nComment: \n'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7ba0f3e10ee0>), output_key='commentator_mood_said'), LLMChain(prompt=PromptTemplate(input_variables=['commentator_mood_said'], template='\nYou are Hyde, the moderator of an online forum, you are strict and will not tolerate any negative comments. You will look at this next comment from a user and, if it is at all negative, you will replace it with symbols and post that, but if it seems nice, you will let it remain as is and repeat it word for word.\nOriginal comment: {commentator_mood_said}\nEdited comment:\n'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_gene